# 「行列計算アルゴリズム 第10章 微分方程式」用ノートブック

# ■メモ

### 概要
- 第10章「微分方程式」で使用するノートブックです。
- 各セルを順に実行することで、本書に記載のとおり実行結果が出力されます。
- 動作確認はJulia 1.10.0 で行っています。

### 使用するスクリプト（.jl）ファイル
- MCA_ode_euler.jl
- MCA_ode_implicit_euler.jl
- MCA_ode_expm.jl
- MCA_ode_coupled.jl
- MCA_pde_diffusion.jl
- MCA_pde_wave.jl
- MCA_pde_poisson.jl
- MCA_pde_schrodinger.jl
- MCA_linsolve_cg.jl（第4章で作成したもの）

### 事前にインストールが必要なパッケージ
- Plots

# ■プログラム

## 10.1. 行列計算の応用としての常微分方程式

### 10.1.1. 線形方程式を利用した常微分方程式の求解

In [ ]:
include("MCA_ode_euler.jl")
include("MCA_ode_implicit_euler.jl");

In [ ]:
m = 1; k = 1; c = 0.1; omega = 0.7; p = 0.1; theta = pi/2
A = [0 1; -k/m -c/m]
MCA_f(t) = [0, p*cos(omega*t + theta)/m]
h = 0.01; tmax = 100; x = [1,0];

In [ ]:
xvece, tvece = MCA_ode_euler(A,MCA_f,x,h,tmax)
xveci, tveci = MCA_ode_implicit_euler(A,MCA_f,x,h,tmax);

In [ ]:
using Plots
plot(tvece,xvece[:,1])
plot!(tveci,xveci[:,1])

### 10.1.2 行列関数を利用した斉次方程式の求解

In [ ]:
include("MCA_ode_expm.jl");

In [ ]:
p = 0                             # f(t) = 0に設定
h = 1e-5; T = 100; x0 = [1,0];

In [ ]:
using Random
Random.seed!(1234)
k = 10
ind = rand(1:Int(round(T/h)),k); t = h * ind     # ランダムに時刻 t を設定
xvece, tvece = MCA_ode_euler(A,MCA_f,x,h,T);

In [ ]:
xt = MCA_ode_expm(A,x0,t)
println("error : ", norm(xt[:,1] - xvece[ind,1]))

### 10.1.3 固有値計算を利用した連成振動問題の求解

In [ ]:
include("MCA_ode_coupled.jl");

In [ ]:
m = [1,0.5]; k = [5,1,2]
x0 = [1,-0.2]
t = range(0,100,1000);

In [ ]:
xt = MCA_ode_coupled(m,k,x0,t);

In [ ]:
using Plots
plt1 = plot(t,xt[1,:])
plt2 = plot(t,xt[2,:])
plot(plt1,plt2,layout = (2,1))

## 10.2 行列計算の応用としての偏微分方程式

### 10.2.1 線形方程式を利用した拡散方程式の求解

In [ ]:
include("MCA_pde_diffusion.jl");

In [ ]:
lambda = 1; c = 10; rho = 10; alpha = lambda/(c*rho)
L = 1; N = 100; h = 0.01; tmax = 10
ux0 = 0; uxL = 100; dx = 1/(N+1); u0 = 100 * ((dx*vec(1:N)).^5);

In [ ]:
u = MCA_pde_diffusion(alpha,L,u0,ux0,uxL,h,tmax);

In [ ]:
using Plots
plot(u0)
for i = 1:10
    plot!(u[i*100,:])
end
plot!()

### 10.2.2 線形方程式を利用した波動方程式の求解

In [ ]:
include("MCA_pde_wave.jl");

In [ ]:
T = 10; rho = 0.1; c = sqrt(T/rho)
L = 1; N = 100; h = 0.01; tmax = 1
dx = L / (N+1); u0 = zeros(N); v0 = dx:dx:N*dx;

In [ ]:
uvec,vvec,tvec = MCA_pde_wave(L,c,u0,v0,h,tmax);

In [ ]:
plot(u0)
for i = 1:10
    plot!(uvec[i*2,:])
end
plot!()

### 10.2.3 線形方程式を利用したポアソン方程式の求解

In [ ]:
# 第4章で作成した MCA_linsolve_cg.jl を読み込む
dir = readdir("../", join=true)
include(dir[4]*"/MCA_linsolve_cg.jl")
include("MCA_pde_poisson.jl");

In [ ]:
L = 1; N = 50;

In [ ]:
# f = -rho(x,y)/eps_0 の設定
c = [0.2,0.4]; r = 0.1; fij = -1e+2
f = zeros(N,N);
dx = dy = L / (N+1);
for i = 1:N
    for j = 1:N
        xi = i*dx; yj = j*dy
        if (xi-c[1])^2 + (yj-c[2])^2 < r^2
            f[i,j] = fij
        end
    end
end
f = vec(f);

In [ ]:
u = MCA_pde_poisson(L,N,f);

In [ ]:
using Plots
contour(u')

### 10.2.4 固有値計算を利用したシュレーディンガー方程式の求解

In [ ]:
include("MCA_pde_schrodinger.jl");

In [ ]:
L = 1; N = 50; hm = 0.01;

In [ ]:
# ポテンシャルエネルギー V(x) の設定
V = zeros(N)
for i = 1:N
    xi = L/(N+1)*i
    V[i] = (xi-L/2)^2
end

In [ ]:
E, U = MCA_pde_schrodinger(L,N,V,hm);

In [ ]:
print("Energy: ", E[1:3])

In [ ]:
using Plots
plot(U[:,1])
plot!(U[:,2])
plot!(U[:,3])